# 1. Download the [dog heart datasets](https://github.com/YoushanZhang/Dog-Cardiomegaly) (10 points) resize image to (224, 224)

### (1). Create a train data loader that returns image arrays and labels
### (2). Create a test data loader that returns image arrays and file names (Note validation data is not used here, it is unlabeled test dataset, DO NOT USE VALIDATION dataset for this part)
### (3). Print image arrays, labels and file names dimensions 

In [1]:
import os
from typing import List, Dict, Tuple

from PIL import Image

import torch
import torch.utils
import torchvision
from torch.utils.data import Dataset, DataLoader

First, create Dataset classes for labeled dataset:

In [2]:
class DogHeartLabeledDataset(Dataset):

    def __init__(self, data_root: str) -> None:
        self.data_root: str = data_root
        self.classes: List[str] = os.listdir(data_root)
        self.class_to_idx: Dict[str, int] = {cls_name: i for i, cls_name in enumerate(self.classes)}

        self.transformation = torchvision.transforms.Compose([
            torchvision.transforms.Grayscale(),
            torchvision.transforms.Resize((224, 224)),
            torchvision.transforms.ToTensor(),
        ])

        self.filenames: List[str] = []
        self.filepaths: List[str] = []
        self.labels: List[int] = []

        for class_name in self.classes:
            path: str = os.path.join(data_root, class_name)
            for filename in os.listdir(path):
                self.filenames.append(filename)
                self.filepaths.append(os.path.join(path, filename))
                self.labels.append(self.class_to_idx[class_name])

    def __len__(self) -> int:
        return len(self.filenames)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, str]:
        filename: str = self.filenames[idx]
        filepath: str = self.filepaths[idx]
        image: Image = Image.open(filepath)
        label: torch.Tensor = torch.tensor(self.labels[idx])
        tensor: torch.Tensor = self.transformation(image)
        tensor = tensor.squeeze(0)
        return tensor, label, filename

Create Dataset class for unlabeled dataset:

In [3]:
class DogHearUnlabeledDataset(Dataset):

    def __init__(self, data_root: str) -> None:
        self.data_root: str = data_root
        self.transformation = torchvision.transforms.Compose([
            torchvision.transforms.Grayscale(),
            torchvision.transforms.Resize((224, 224)),
            torchvision.transforms.ToTensor(),
        ])
        self.filenames: List[str] = os.listdir(self.data_root)
    
    def __len__(self) -> int:
        return len(self.filenames)
    
    def __getitem__(self, idx) -> Tuple[torch.Tensor, str]:
        filename: str = self.filenames[idx]
        image: Image = Image.open(os.path.join(self.data_root, filename))
        tensor: torch.Tensor = self.transformation(image)
        tensor = tensor.squeeze(0)
        return tensor, filename

In [4]:
train_dataset = DogHeartLabeledDataset(data_root='Dog_heart/Train')
valid_dataset = DogHeartLabeledDataset(data_root='Dog_heart/Valid')
test_dataset = DogHearUnlabeledDataset(data_root='Test')

train_dataloader = DataLoader(dataset=train_dataset, batch_size=len(train_dataset), shuffle=True)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=len(valid_dataset), shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)

Dimensions of train dataset:

In [5]:
train_images, train_labels, train_filenames = next(iter(train_dataloader))
print(train_images.shape)
print(train_labels.shape)
print(len(train_filenames))

torch.Size([1400, 224, 224])
torch.Size([1400])
1400


Dimensions of validation dataset:

In [6]:
valid_images, valid_labels, valid_filenames = next(iter(valid_dataloader))
print(valid_images.shape)
print(valid_labels.shape)
print(len(valid_filenames))

torch.Size([200, 224, 224])
torch.Size([200])
200


Dimensions of test dataset:

In [7]:
test_images, test_filenames = next(iter(test_dataloader))
print(test_images.shape)
print(len(test_filenames))

torch.Size([400, 224, 224])
400


# 2. Extract features of training and test images using HOG (20 points)
Please print the size of extracted features, e.g., training features: 1400 * d, test features: 400 *d

In [8]:
from abc import ABC, abstractmethod
from typing import List

from datasets import DogHeartLabeledDataset

import numpy as np
import cv2
import skimage.feature

import torch
import torch.utils

First, create an abstract `FeatureExtractor` class

In [9]:
class FeatureExtractor(ABC):

    @abstractmethod
    def __call__(self, image_array: np.ndarray) -> np.ndarray:
        pass

Create the `HOG` class:

In [10]:
class HOG(FeatureExtractor):

    def __init__(self, channel_axis: int = None) -> None:
        self.channel_axis: int = channel_axis

    def __call__(self, image_array: np.ndarray) -> np.ndarray:
        return skimage.feature.hog(image=image_array)

In [11]:
train_dataset = DogHeartLabeledDataset(data_root='Dog_heart/Train')
valid_dataset = DogHeartLabeledDataset(data_root='Dog_heart/Valid')

hog = HOG()

HOG feature extractor on Train:

In [12]:
train_hog_features: List[np.ndarray] = []
for image, label, filename in train_dataset:
    hog_feature = hog(image_array=image.numpy())
    train_hog_features.append(hog_feature)

train_hog_features = np.stack(arrays=train_hog_features, axis=0)
print(train_hog_features.shape)

(1400, 54756)


HOG feature extractor on Validation:

In [13]:
valid_hog_features: List[np.ndarray] = []
for image, label, filename in valid_dataset:
    hog_feature = hog(image_array=image.numpy())
    valid_hog_features.append(hog_feature)

valid_hog_features = np.stack(arrays=valid_hog_features, axis=0)
print(valid_hog_features.shape)

(200, 54756)


HOG feature extractor on Test:

In [14]:
test_hog_features: List[np.ndarray] = []
for image, filename in test_dataset:
    hog_feature = hog(image_array=image.numpy())
    test_hog_features.append(hog_feature)

test_hog_features = np.stack(arrays=test_hog_features, axis=0)
print(test_hog_features.shape)

(400, 54756)


# 3. Extract features of training and test images using SIFT (20 points)
Please print the size of extracted features, e.g., training features: 1400 * d, test features: 400 *d

Create the `SIFT` class:

In [15]:
class SIFT(FeatureExtractor):

    def __init__(self, n_features: int):
        self.n_features: int = n_features
        self.sift: cv2.SIFT = cv2.xfeatures2d.SIFT_create()

    def __call__(self, image_array: np.ndarray) -> np.ndarray:
        keypoints: List[cv2.KeyPoint]
        descriptors: np.ndarray
        keypoints, descriptors = self.sift.detectAndCompute(
            image=(image_array * 255).astype(np.uint8), 
            mask=None,
        )   # (keypoints, descriptors)

        assert len(keypoints) == descriptors.shape[0]
        indices: np.ndarray = np.argsort([kp.response for kp in keypoints])[::-1][:self.n_features]

        keypoints: List[cv2.KeyPoint] = [keypoints[idx] for idx in indices]
        features: np.ndarray = descriptors[indices]
        if features.shape[0] < self.n_features:
            padding = np.zeros(shape=(self.n_features - features.shape[0], features.shape[1]))
            features = np.concatenate((features, padding), axis=0)
        
        features: np.ndarray = features.flatten()
        return features

In [16]:
sift = SIFT(n_features=30)

SIFT feature extractor on Train:

In [17]:
train_sift_features: List[np.ndarray] = []
for image, label, filename in train_dataset:
    sift_feature = sift(image_array=image.numpy())
    train_sift_features.append(sift_feature)

train_sift_features = np.stack(arrays=train_sift_features, axis=0)
print(train_sift_features.shape)


(1400, 3840)


SIFT feature extractor on Validation:

In [18]:
valid_sift_features: List[np.ndarray] = []
for image, label, filename in valid_dataset:
    sift_feature = sift(image_array=image.numpy())
    valid_sift_features.append(sift_feature)

valid_sift_features = np.stack(arrays=valid_sift_features, axis=0)
print(valid_sift_features.shape)

(200, 3840)


SIFT feature extractor on Test:

In [19]:
test_sift_features: List[np.ndarray] = []
for image, filename in test_dataset:
    sift_feature = sift(image_array=image.numpy())
    test_sift_features.append(sift_feature)

test_sift_features = np.stack(arrays=test_sift_features, axis=0)
print(test_sift_features.shape)

(400, 3840)


# 4. Extract features of training and test images using SURF (20 points)
Please print the size of extracted features, e.g., training features: 1400 * d, test features: 400 *d

Create the `SURF` class:

In [20]:
class SURF(FeatureExtractor):

    def __init__(self, n_features: int):
        self.n_features: int = n_features
        self.surf: cv2.xfeatures2d_SURF = cv2.xfeatures2d.SURF_create(nOctaves=5)

    def __call__(self, image_array: np.ndarray) -> np.ndarray:
        keypoints: List[cv2.KeyPoint]
        descriptors: np.ndarray
        keypoints, descriptors = self.surf.detectAndCompute(
            image=(image_array * 255).astype(np.uint8), 
            mask=None,
        )   # (keypoints, descriptors)

        assert len(keypoints) == descriptors.shape[0]
        indices: np.ndarray = np.argsort([kp.response for kp in keypoints])[::-1][:self.n_features]

        keypoints: List[cv2.KeyPoint] = [keypoints[idx] for idx in indices]
        features: np.ndarray = descriptors[indices]
        if features.shape[0] < self.n_features:
            padding = np.zeros(shape=(self.n_features - features.shape[0], features.shape[1]))
            features = np.concatenate((features, padding), axis=0)

        features = features.flatten()
        return features

In [21]:
surf = SURF(n_features=30)

SURF feature extractor on Train:

In [22]:
train_surf_features: List[np.ndarray] = []
for image, label, filename in train_dataset:
    surf_feature = surf(image_array=image.numpy())
    train_surf_features.append(surf_feature)

train_surf_features = np.stack(arrays=train_surf_features, axis=0)
print(train_surf_features.shape)

(1400, 1920)


SURF feature extractor on Validation:

In [23]:
valid_surf_features: List[np.ndarray] = []
for image, label, filename in valid_dataset:
    surf_feature = surf(image_array=image.numpy())
    valid_surf_features.append(surf_feature)

valid_surf_features = np.stack(arrays=valid_surf_features, axis=0)
print(valid_surf_features.shape)

(200, 1920)


SURF feature extractor on Test:

In [24]:
test_surf_features: List[np.ndarray] = []
for image, filename in test_dataset:
    surf_feature = surf(image_array=image.numpy())
    test_surf_features.append(surf_feature)

test_surf_features = np.stack(arrays=test_surf_features, axis=0)
print(test_surf_features.shape)

(400, 1920)


# 5. Call SVM and kNN from scikit-learn and train the extracted HOG, SIFT and SURF features, respectively, save three CSV files of test dataset using three features (10 points)

In [25]:
from typing import List
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import torch
from torch.utils.data import Dataset, DataLoader

from feature_extractors import FeatureExtractor
from datasets import DogHeartLabeledDataset, DogHearUnlabeledDataset

Create the class `Predictor` that accepts one `Estimator` and one `FeatureExtractor`

In [26]:
class Predictor:

    def __init__(
        self, 
        model: BaseEstimator,
        feature_extractor: FeatureExtractor,
    ):
        self.model = model
        self.feature_extractor: FeatureExtractor = feature_extractor

    def fit(self, train_dataset: Dataset) -> None:
        train_dataloader = DataLoader(dataset=train_dataset, batch_size=len(train_dataset), shuffle=False)
        
        images: torch.Tensor; labels: torch.Tensor; filenames: List[str]
        images, labels, filenames = next(iter(train_dataloader))
        
        images: np.ndarray = images.numpy()
        labels: np.ndarray = labels.numpy()
        
        train_features: List[np.ndarray] = []
        for image in images:
            feature = self.feature_extractor(image_array=image)
            train_features.append(feature)

        train_features = np.stack(arrays=train_features, axis=0)
        self.model.fit(X=train_features, y=labels)

    def predict(self, test_dataset: Dataset) -> np.ndarray:
        test_dataloader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)
        
        image: torch.Tensor; filenames: List[str]
        images, filenames = next(iter(test_dataloader))

        images: np.ndarray = images.numpy()

        test_features: List[np.ndarray] = []
        for image in images:
            feature = self.feature_extractor(image_array=image).reshape(-1)
            test_features.append(feature)

        predicted_labels: np.ndarray = self.model.predict(X=np.array(test_features))
        prediction_table = pd.DataFrame(
            data={'image': filenames, 'label': predicted_labels}
        )
        prediction_table.to_csv(
            f'{self.model.__class__.__name__}_{self.feature_extractor.__class__.__name__}.csv', 
            header=False, 
            index=False,
        )
        return prediction_table

Load train (labeled) and test (unlabeled) datasets:

In [27]:
train_dataset = DogHeartLabeledDataset(data_root='Dog_heart/Train')
test_dataset = DogHearUnlabeledDataset('Test')

`SVM + HOG`:

In [28]:
svm_hog = Predictor(model=SVC(), feature_extractor=HOG())
svm_hog.fit(train_dataset=train_dataset)
svm_hog.predict(test_dataset=test_dataset)

,image,label
0,1922.png,2
1,1709.png,1
2,1919.png,0
3,1639.png,0
4,1804.png,2
...,...,...
395,1685.png,2
396,1833.png,2
397,1900.png,0
398,1824.png,2


`KNN + HOG`:

In [29]:
knn_hog = Predictor(model=KNeighborsClassifier(n_neighbors=1), feature_extractor=HOG())
knn_hog.fit(train_dataset=train_dataset)
knn_hog.predict(test_dataset=test_dataset)

,image,label
0,1922.png,2
1,1709.png,1
2,1919.png,0
3,1639.png,0
4,1804.png,2
...,...,...
395,1685.png,2
396,1833.png,2
397,1900.png,0
398,1824.png,2


`SVM + SIFT`:

In [30]:
svm_sift = Predictor(model=SVC(), feature_extractor=SIFT(n_features=30))
svm_sift.fit(train_dataset=train_dataset)
svm_sift.predict(test_dataset=test_dataset)

,image,label
0,1922.png,0
1,1709.png,1
2,1919.png,0
3,1639.png,2
4,1804.png,0
...,...,...
395,1685.png,2
396,1833.png,2
397,1900.png,0
398,1824.png,0


`KNN + SIFT`:

In [31]:
knn_sift = Predictor(model=KNeighborsClassifier(n_neighbors=1), feature_extractor=SIFT(n_features=30))
knn_sift.fit(train_dataset=train_dataset)
knn_sift.predict(test_dataset=test_dataset)

,image,label
0,1922.png,0
1,1709.png,1
2,1919.png,0
3,1639.png,0
4,1804.png,1
...,...,...
395,1685.png,2
396,1833.png,2
397,1900.png,0
398,1824.png,0


`SVM + SURF`:

In [32]:
svm_surf = Predictor(model=SVC(), feature_extractor=SURF(n_features=30))
svm_surf.fit(train_dataset=train_dataset)
svm_surf.predict(test_dataset=test_dataset)

,image,label
0,1922.png,0
1,1709.png,0
2,1919.png,0
3,1639.png,0
4,1804.png,2
...,...,...
395,1685.png,2
396,1833.png,2
397,1900.png,0
398,1824.png,2


`KNN + SURF`:

In [33]:
knn_surf = Predictor(model=KNeighborsClassifier(n_neighbors=1), feature_extractor=SURF(n_features=30))
knn_surf.fit(train_dataset=train_dataset)
knn_surf.predict(test_dataset=test_dataset)

,image,label
0,1922.png,0
1,1709.png,0
2,1919.png,0
3,1639.png,0
4,1804.png,2
...,...,...
395,1685.png,0
396,1833.png,2
397,1900.png,2
398,1824.png,0


# 6. Report the accuracy using Dog_X_ray_classfication_accuracy software, please attach the results image here (20 points)

### (1). SVM and 1NN using HOG features

`SVM + HOG`:
<br>
<img src="https://raw.githubusercontent.com/hiepdang-ml/DLAssignment3/master/SVC_HOG.png" style="width:50%;">


`KNN + HOG`:
<br>
<img src="https://raw.githubusercontent.com/hiepdang-ml/DLAssignment3/master/KNN_HOG.png" style="width:50%;">

### (2). SVM and 1NN using SIFT features

`SVM + SIFT`:
<br>
<img src="https://raw.githubusercontent.com/hiepdang-ml/DLAssignment3/master/SVC_SIFT.png" style="width:50%;">


`KNN + SIFT`:
<br>
<img src="https://raw.githubusercontent.com/hiepdang-ml/DLAssignment3/master/KNN_SIFT.png" style="width:50%;">

### (3). SVM and 1NN using SURF features

`SVM + SURF`:
<br>
<img src="https://raw.githubusercontent.com/hiepdang-ml/DLAssignment3/master/SVC_SURF.png" style="width:50%;">


`KNN + SIFT`:
<br>
<img src="https://raw.githubusercontent.com/hiepdang-ml/DLAssignment3/master/KNN_SURF.png" style="width:50%;">

---